In [226]:
import pandas as pd
import os
import psycopg2
from psycopg2 import OperationalError

# Definición de la función para establecer la conexión a PostgreSQL.


def connect_to_postgres():
    try:
        # Conexión a la base de datos PostgreSQL.
        connection = psycopg2.connect(
            dbname="Prueba_diego",
            user="postgres",
            password="Colombia2024*",
            host="localhost",
            port="5432"
        )

        # Establecer datestyle a 'ISO, MDY' para el proceso o casteo de la fecha.
        cursor = connection.cursor()
        cursor.execute("SET datestyle TO 'ISO, MDY';")
        cursor.close()

        print("Conexión exitosa a PostgreSQL")
        return connection
    except OperationalError as e:
        print(f"Error al intentar conectarse a PostgreSQL: {e}")
        return None

# Definición de la función para cargar los datos en la base de datos PostgreSQL.


def load_data_to_postgres(connection, merged_df):
    try:
        cursor = connection.cursor()
        count = 0
        total_price = 0
        min_price = float('inf')
        max_price = float('-inf')

        # Insertar datos de merged_df en la tabla prueba.Prueba_diego_ingesta.
        for index, row in merged_df.iterrows():
            cursor.execute("INSERT INTO prueba.\"Prueba_diego_ingesta\" (timestamp, price, user_id) VALUES (%s, %s, %s)",
                           (row['timestamp'], row['price'], row['user_id']))
            count += 1
            price = row['price']
            total_price += price
            if price < min_price:
                min_price = price
            if price > max_price:
                max_price = price

        connection.commit()
        print("Datos insertados correctamente.")
        print(f"Número total de filas cargadas: {count}")
        print(f"Valor medio del campo 'price': {total_price / count}")
        print(f"Valor mínimo del campo 'price': {min_price}")
        print(f"Valor máximo del campo 'price': {max_price}")

    except (psycopg2.Error, Exception) as e:
        print(f"Error al insertar datos en PostgreSQL: {e}")
        connection.rollback()
    finally:
        if cursor:
            cursor.close()


############################### Directorio donde se encuentra el archivo CSV.###############################
directory = "C:/Users/Diego/Documents/HOJA DE VIDA/Pragma/PRUEBA INGENIERO DE DATOS - DIEGO CHINCHILLA/dataPruebaDataEngineer"

# Obtener la lista de archivos CSV en el directorio (excepto validation.csv).
csv_files = [file for file in os.listdir(
    directory) if file.endswith(".csv") and file != "validation.csv"]

############################### Crear una lista para almacenar los DataFrames cargados de cada archivo CSV.###############################
dataframes = []

############################### Iterar sobre los archivos CSV y cargarlos en DataFramess.###############################
for file in csv_files:
    file_path = os.path.join(directory, file)
    # Al leer el CSV, especifica que la coma (',') es el delimitador de campos.
    df = pd.read_csv(file_path, delimiter=',')
    dataframes.append(df)

############################### Concatenar los DataFrames en uno solo. ###############################
merged_df = pd.concat(dataframes, ignore_index=True)

############################### Convertir la columna 'price' a tipo string y luego reemplazar el punto decimal por coma.###############################
merged_df['price'] = merged_df['price'].astype(str).str.replace('.', ',')

############################### Eliminar cualquier caracter despues de la coma en la columna 'price' y convertirla a tipo numérico.###############################
merged_df['price'] = merged_df['price'].apply(lambda x: x.split(',')[0])
merged_df['price'] = pd.to_numeric(merged_df['price'], errors='coerce')

############################### Rellenar los valores nulos en la columna 'price' con ceros. ###############################
merged_df['price'].fillna(0, inplace=True)

############################### Verificar el DataFrame resultante. ###############################
print(merged_df.head(500))

############################### Sumar la columna 'price' ###############################
total_price = merged_df['price'].sum()
print("Total de precios:", total_price)

############################### Conectarse a PostgreSQL. ###############################
connection = connect_to_postgres()

############################### Cargar los datos en PostgreSQL. ###############################
if connection:
    load_data_to_postgres(connection, merged_df)

     timestamp  price  user_id
0    1/10/2012   50.0        9
1    1/11/2012   87.0       10
2    1/12/2012   64.0        7
3    1/13/2012   20.0       10
4    1/14/2012   14.0       10
..         ...    ...      ...
138  5/27/2012   61.0        2
139  5/28/2012   67.0        8
140  5/29/2012   39.0       10
141  5/30/2012   57.0        8
142  5/31/2012   33.0        6

[143 rows x 3 columns]
Total de precios: 8046.0
Conexión exitosa a PostgreSQL
Datos insertados correctamente.
Número total de filas cargadas: 143
Valor medio del campo 'price': 56.26573426573427
Valor mínimo del campo 'price': 0.0
Valor máximo del campo 'price': 100.0


C:\Users\Diego\AppData\Local\Temp\ipykernel_15048\3478628176.py:83: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  merged_df['price'] = merged_df['price'].astype(str).str.replace('.', ',')
